In [1]:
import gzip
import struct
import os

old_init = gzip.GzipFile.__init__

def init_mod(self, filename=None, mode=None, 
             compresslevel=9, fileobj=None, mtime=None, extra=None):
    self.extra = extra
    old_init(self, filename, mode, compresslevel, fileobj, mtime)

gzip.GzipFile.__init__ = init_mod

def mod_write_gzip_header(self):
    self.fileobj.write(b'\037\213')             # magic header
    self.fileobj.write(b'\010')                 # compression method
    flags = 0
    if self.extra:
        flags = gzip.FEXTRA
    self.fileobj.write(chr(flags).encode('latin-1'))
    self.fileobj.write(struct.pack('<I', 0))
    self.fileobj.write(b'\002')
    self.fileobj.write(b'\377')
    if self.extra:
        self.fileobj.write(struct.pack('<H', len(self.extra)))
        self.fileobj.write(self.extra)

def read_gzip_extra(filename):
    f = open(filename, 'rb')
    magic = f.read(2)
    if magic != b'\x1f\x8b':
        return None
    f.seek(3)
    flags = f.read(1)
    flags = ord(flags)
    if (flags & 4) != 4:
        return None
    f.seek(10)
    extra_len = f.read(2)
    extra_len, = struct.unpack('<H', extra_len)
    extra = f.read(extra_len)
    return extra
        
gzip.GzipFile._write_gzip_header = mod_write_gzip_header

magic = b'\xab\x57'

def bytes_2_bits(raw):
    ret = []
    for b in raw:
        for i in range(8):
            ret.append(b&1)
            b = b >> 1
    return ret

def bits_2_bytes(bits):
    ret = []
    while bits:
        a, bits = bits[:8], bits[8:]
        v = 0
        a.reverse()
        for bit in a:
            v = v | bit
            v = v << 1
        v = v >> 1
        ret.append(v)
    return bytes(ret)

def water_mark(raw, info):
    if len(raw) != 8452:
        return None
    if len(info) > 500:
        return None
    bits = bytes_2_bits(magic+struct.pack('<H', len(info))+info+magic)
    ret = bytearray(raw)
    for i in range(len(bits)):
        ret[i*2] = ret[i*2] & 0xFE | bits[i]
    return bytes(ret)

def extract_water_mark(raw):
    if len(raw) != 8452:
        return None
    bits = [(raw[i*2] & 1) for i in range(8452//2)]
    a = bits_2_bytes(bits)
    if a[:2] != magic:
        return None
    info_len, = struct.unpack('<h', a[2:4])
    if a[4+info_len:6+info_len] != magic:
        return None
    return a[4:4+info_len]

def patch_file_out(filename, info):
    content = gzip.GzipFile(filename, 'rb').read()
    f = gzip.GzipFile(filename, 'wb', extra=info)
    f.write(content)
    f.close()

def patch_file_in(filename, info):
    content = gzip.GzipFile(filename, 'rb').read()
    content = water_mark(content, info)
    gzip.GzipFile(filename, 'wb').write(content)
    
def get_water_mark_in(filename):
    content = gzip.GzipFile(filename, 'rb').read()
    return extract_water_mark(content)

def get_water_mark_out(filename):
    return read_gzip_extra(filename)

def patch_file_in_pwd(info):
    files = os.listdir('.')
    for f in files:
        if f.endswith('.terrain'):
            patch_file_in(f, info)

In [3]:
patch_file_in('/tmp/25093.terrain', b'kedacom')

In [2]:
a = gzip.GzipFile('/tmp/test.gz', 'wb', extra=b'testing')
a.write(b'abcdef'*30)
a.close()

In [28]:
a = b'abc'
b = bytearray(a)

In [3]:
get_water_mark_out('/tmp/25093.terrain')

In [36]:
bytes(b)

b'\xeabc'

In [6]:
import os

os.listdir?

In [85]:
magic = b'\xab\x57'

def bytes_2_bits(raw):
    ret = []
    for b in raw:
        for i in range(8):
            ret.append(b&1)
            b = b >> 1
    return ret

def bits_2_bytes(bits):
    ret = []
    while bits:
        a, bits = bits[:8], bits[8:]
        v = 0
        a.reverse()
        for bit in a:
            v = v | bit
            v = v << 1
        v = v >> 1
        ret.append(v)
    return bytes(ret)

def water_mark(raw, info):
    if len(raw) != 8452:
        return None
    if len(info) > 500:
        return None
    bits = bytes_2_bits(magic+struct.pack('<H', len(info))+info+magic)
    ret = bytearray(raw)
    for i in range(len(bits)):
#         print(ret[i*2])
        ret[i*2] = ret[i*2] & 0xFE | bits[i]
#         print(ret[i*2], '===')
    return bytes(ret)

def extract_water_mark(raw):
    if len(raw) != 8452:
        return None
    bits = [(raw[i*2] & 1) for i in range(8452//2)]
    a = bits_2_bytes(bits)
    if a[:2] != magic:
        return None
    info_len, = struct.unpack('<h', a[2:4])
    if a[4+info_len:6+info_len] != magic:
        return None
    return a[4:4+info_len]

In [63]:
bytes([12, 255])

b'\x0c\xff'

In [79]:
bits_2_bytes(bytes_2_bits(b'hessdfsdoosidfj\x99ok'))

b'hessdfsdoosidfj\x99ok'

In [82]:
bb = water_mark(b'A'*8452, b'kedacom')

In [86]:
extract_water_mark(bb)

b'kedacom'